## Import and Mount

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.autograd import Variable, grad
from torch.optim.lr_scheduler import StepLR, ExponentialLR
from torch.utils import data
from torch.distributions import MultivariateNormal
from torch.nn.utils import weight_norm
from torchvision import models
import torchvision.utils as vutils

try:
    from torchinfo import summary
except ImportError:
    !pip install torchinfo
    from torchinfo import summary

try:
    import mat73
except ImportError:
    !pip install mat73

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import os
import time
import math
from collections import defaultdict
from timeit import default_timer
import random

from google.colab import drive
drive.mount('/content/gdrive')

work_dir = './RGA-FNO-HT-INV'

os.chdir(work_dir)
!pwd

from plot_utils.plotslib import *
from FNO.utils import _get_act, add_padding2, remove_padding2
from FNO.utilities3 import UnitGaussianNormalizer, LpLoss
from FNO.basics import SpectralConv2d

In [ ]:
# Set random seed for reproducibility
manualSeed = 999
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)
torch.cuda.manual_seed(manualSeed)

def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        torch.set_default_tensor_type(torch.cuda.FloatTensor)
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

device0 = try_gpu()
print(device0)

# Load data

In [ ]:
data_dir = work_dir + '/Data/'

model_dir = work_dir + '/models/'
fig_dir = work_dir + '/figs/'

try:
    os.makedirs(model_dir)
    os.makedirs(fig_dir)
except: Exception

In [ ]:
fname = "Exponential_steady_state"
filename = data_dir + fname + "_fields.npz"
with np.load(filename) as npzfile:
    realizations = npzfile['realizations']
    alpha = npzfile['alpha']
    Z = npzfile['Z']
    Q = npzfile['Q'][0]
    pump_id_list = npzfile['pump_id_list']
    for k in ['lx', 'ly', 'sigma2']:
        if k in npzfile.keys(): print(k, npzfile[k])
filename = data_dir + fname + "_heads.npy"
all_heads = np.load(filename, mmap_mode='r')
print(Q)
print(pump_id_list)
print(alpha.shape)
print(Z.shape)

print(realizations.shape)
print(all_heads.shape)

NR=realizations.shape[0]


# Experimental Domain
## Unit, Zero-centered

In [ ]:
############### define domain with (0,0) at center ######
nx = ny = int(math.sqrt(all_heads.shape[1]))
dx_real = 5.0/(nx/64.0)
dt_real = 0.1

Lox, Loy = 1, 1
dx, dy = Lox/nx, Loy/ny

x = np.arange((-Lox/2+dx/2),(Lox/2),dx)
y = np.arange((-Lox/2+dx/2),(Lox/2),dy)

Xm, Ym = np.meshgrid(x,y)

X_star = np.hstack((Xm.flatten()[:,None], Ym.flatten()[:,None]))


# Fourier Neural Operator class

In [ ]:
class FNO2d(nn.Module):
    def __init__(self, modes1, modes2,
                 width=64, fc_dim=128,
                 layers=None,
                 in_dim=3, out_dim=1,
                 act='gelu',
                 pad_ratio=[0., 0.]):
        super(FNO2d, self).__init__()
        if isinstance(pad_ratio, float):
            pad_ratio = [pad_ratio, pad_ratio]
        else:
            assert len(pad_ratio) == 2, 'Cannot add padding in more than 2 directions'
        self.modes1 = modes1
        self.modes2 = modes2

        self.pad_ratio = pad_ratio
        if layers is None:
            self.layers = [width] * (len(modes1) + 1)
        else:
            self.layers = layers
        self.fc0 = nn.Linear(in_dim, layers[0])

        self.sp_convs = nn.ModuleList([SpectralConv2d(
            in_size, out_size, mode1_num, mode2_num)
            for in_size, out_size, mode1_num, mode2_num
            in zip(self.layers, self.layers[1:], self.modes1, self.modes2)])

        self.ws = nn.ModuleList([nn.Conv1d(in_size, out_size, 1)
                                 for in_size, out_size in zip(self.layers, self.layers[1:])])

        self.fc1 = nn.Linear(layers[-1], fc_dim)
        self.fc2 = nn.Linear(fc_dim, layers[-1])
        self.fc3 = nn.Linear(layers[-1], out_dim)
        self.act = _get_act(act)

    def forward(self, x):
        size_1, size_2 = x.shape[1], x.shape[2]
        if max(self.pad_ratio) > 0:
            num_pad1 = [round(i * size_1) for i in self.pad_ratio]
            num_pad2 = [round(i * size_2) for i in self.pad_ratio]
        else:
            num_pad1 = num_pad2 = [0.]

        length = len(self.ws)
        batchsize = x.shape[0]
        grid = self.get_grid(x.shape, x.device)
        x = torch.cat((x, grid), dim=-1)
        x = self.fc0(x)
        x = x.permute(0, 3, 1, 2)   # B, C, X, Y
        x = add_padding2(x, num_pad1, num_pad2)
        size_x, size_y = x.shape[-2], x.shape[-1]

        for i, (speconv, w) in enumerate(zip(self.sp_convs, self.ws)):
            x1 = speconv(x)
            x2 = w(x.view(batchsize, self.layers[i], -1)).view(batchsize, self.layers[i+1], size_x, size_y)
            x = x1 + x2
            if i != length - 1:
                x = self.act(x)
        x = remove_padding2(x, num_pad1, num_pad2)
        x = x.permute(0, 2, 3, 1)
        x = self.fc1(x)
        x = self.act(x)
        x = self.fc2(x)
        x = self.act(x)
        x = self.fc3(x)
        return x

    def get_grid(self, shape, device):
        batchsize, size_x, size_y = shape[0], shape[1], shape[2]
        gridx = torch.tensor(np.linspace(0, 1, size_x), dtype=torch.float)
        gridx = gridx.reshape(1, size_x, 1, 1).repeat([batchsize, 1, size_y, 1])
        gridy = torch.tensor(np.linspace(0, 1, size_y), dtype=torch.float)
        gridy = gridy.reshape(1, 1, size_y, 1).repeat([batchsize, size_x, 1, 1])
        return torch.cat((gridx, gridy), dim=-1).to(device)

# Training data
## sample
## normalization

In [ ]:
heads = all_heads[:,:,-1,:].reshape((NR,nx,ny,-1))
hmean = np.mean(heads,0)
hstd = np.std(heads,0)

Kmean = np.mean(np.exp(realizations),0)
Kstd = np.std(np.exp(realizations),0)


In [ ]:
r = 2                     # spatial sampling steps
h = int(((nx - 1)/r) + 1) # sampled data resolution

pid = [0,1,2,3,4]         # pumping well id
x_normalizer = UnitGaussianNormalizer(torch.ones((0,)))
x_normalizer.mean = torch.tensor(Kmean[::r,::r],dtype=torch.float32)
x_normalizer.std = torch.tensor(Kstd[::r,::r],dtype=torch.float32)

y_normalizer = UnitGaussianNormalizer(torch.ones((0,)))
y_normalizer.mean = torch.tensor(hmean[::r,::r,pid],dtype=torch.float32).squeeze(-1)
y_normalizer.std = torch.tensor(hstd[::r,::r,pid],dtype=torch.float32).squeeze(-1)

In [ ]:
ntrain = 500
ntest = 100

################################################################
# load data and data normalization
################################################################
x_train = torch.tensor(np.exp(realizations[:ntrain,::r,::r]), dtype=torch.float32)
x_test = torch.tensor(np.exp(realizations[-ntest:,::r,::r]), dtype=torch.float32)

y_train = torch.tensor(heads[:ntrain,...,pid][:,::r,::r], dtype=torch.float32).squeeze(-1)
y_test = torch.tensor(heads[-ntest:,...,pid][:,::r,::r], dtype=torch.float32).squeeze(-1)

x_train = x_normalizer.encode(x_train)
x_test = x_normalizer.encode(x_test)

y_train = y_normalizer.encode(y_train)
y_test = y_normalizer.encode(y_test)

x_train = x_train.unsqueeze(-1)
x_test = x_test.unsqueeze(-1)

# FNO Model Init
## Configuration and Initialization

In [ ]:
config = defaultdict(dict)
config['model'] = {
    'modes1':[20]*4,
    'modes2':[20]*4,
    'fc_dim':128,
    'layers':[64]*5,
    'pad_ratio':[1,1],
    'out_dim':5,
    'act':'gelu'
}
model = FNO2d(
    modes1=config['model']['modes1'],
    modes2=config['model']['modes2'],
    fc_dim=config['model']['fc_dim'],
    layers=config['model']['layers'],
    pad_ratio=config['model']['pad_ratio'],
    out_dim=config['model']['out_dim'],
    act=config['model']['act']
).to(device0)

summary(model, (1,32,32,1))


# Training Loop

In [ ]:
batch_size = 10
learning_rate = 0.001

# data loader
train_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_train, y_train), \
    batch_size=batch_size,\
    shuffle=True,\
    generator=torch.Generator(device=device0)
)

test_loader = torch.utils.data.DataLoader(
    torch.utils.data.TensorDataset(x_test, y_test),\
    batch_size=batch_size,\
    shuffle=False,\
    generator=torch.Generator(device=device0)
)

# optimizer and scheduler
epochs = 500
iterations = epochs*(ntrain//batch_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=iterations)

# loss function
myloss = LpLoss(size_average=False)

for ep in range(epochs):
    #######################
    # training step
    #######################
    model.train()
    t1 = default_timer()
    train_l2 = 0
    for x, y in train_loader:
        x, y = x.to(device0), y.to(device0)
        # print(x.shape, y.shape)
        optimizer.zero_grad()
        out = model(x).squeeze(-1)

        loss = myloss(out.reshape(batch_size,-1), y.reshape(batch_size,-1))
        loss.backward()

        optimizer.step()
        scheduler.step()
        train_l2 += loss.item()

    #######################
    # validation loss
    #######################
    model.eval()
    test_l2 = 0.0
    with torch.no_grad():
        for x, y in test_loader:
            x, y = x.to(device0), y.to(device0)
            out = model(x).squeeze(-1)
            test_l2 += myloss(out.reshape(batch_size,-1), y.reshape(batch_size,-1)).item()

    train_l2/= ntrain
    test_l2 /= ntest

    t2 = default_timer()

    print("Epoch#: %d" % ep, end="\t")
    print("Time: %.4f" % (t2-t1), end="\t")
    print("LR: %f" % optimizer.param_groups[0]['lr'], end="\t")
    print("Loss:", end=" ")
    print("[Train %.4f]" % (train_l2), end="\t")
    print("[Valid: %.4f]" % (test_l2), end="\n")

# Trained Model Evaluation

## Evaluate on finer resolution

In [ ]:
x_normalizer_recover = UnitGaussianNormalizer(torch.ones((0,)))
x_normalizer_recover.mean = torch.tensor(Kmean[...],dtype=torch.float32)
x_normalizer_recover.std = torch.tensor(Kstd[...],dtype=torch.float32)

y_normalizer_recover = UnitGaussianNormalizer(torch.ones((0,)))
y_normalizer_recover.mean = torch.tensor(hmean[...,pid],dtype=torch.float32).squeeze(-1)
y_normalizer_recover.std = torch.tensor(hstd[...,pid],dtype=torch.float32).squeeze(-1)

# Selection 10 unseen realizations
shift = 550
x_recover = torch.tensor(np.exp(realizations[shift:shift+10,...]),dtype=torch.float32)
ys = torch.tensor(heads[shift:shift+10,:,:,pid], dtype=torch.float32).squeeze(-1)

In [ ]:
model.eval()

with torch.no_grad():
    outs = model(x_normalizer_recover.encode(x_recover).unsqueeze(-1))
    outs = y_normalizer_recover.decode(outs)


## Plot Results

In [ ]:
preds = outs[...].detach().cpu().numpy()
refs = ys[...].detach().cpu().numpy()
fig = plot_forward_operator(realizations[shift:shift+10,...], preds[...,0], refs[...,0], cmaplnK='jet')

# Save Trained Model

In [ ]:
# torch.save(model.state_dict(), model_dir+fname+'_FNO_HT.pt')

In [ ]:
# fname = "Exponential_steady_state"
# print(fname+'_FNO_HT.pth')
# model.load_state_dict(torch.load(model_dir+fname+'_FNO_HT.pt', map_location=device0))
